<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#3.3-线性回归的简洁实现" data-toc-modified-id="3.3-线性回归的简洁实现-1">3.3 线性回归的简洁实现</a></span><ul class="toc-item"><li><span><a href="#3.3.1-生成数据集" data-toc-modified-id="3.3.1-生成数据集-1.1">3.3.1 生成数据集</a></span></li><li><span><a href="#3.3.2-读取数据" data-toc-modified-id="3.3.2-读取数据-1.2">3.3.2 读取数据</a></span></li><li><span><a href="#3.3.3-定义模型和初始化参数" data-toc-modified-id="3.3.3-定义模型和初始化参数-1.3">3.3.3 定义模型和初始化参数</a></span></li><li><span><a href="#3.3.4-定义损失函数" data-toc-modified-id="3.3.4-定义损失函数-1.4">3.3.4 定义损失函数</a></span></li><li><span><a href="#3.3.5-定义优化算法" data-toc-modified-id="3.3.5-定义优化算法-1.5">3.3.5 定义优化算法</a></span></li><li><span><a href="#3.3.6-训练模型" data-toc-modified-id="3.3.6-训练模型-1.6">3.3.6 训练模型</a></span></li><li><span><a href="#小结" data-toc-modified-id="小结-1.7">小结</a></span></li></ul></li></ul></div>

# 3.3 线性回归的简洁实现

随着深度学习框架的发展，开发深度学习应用变得越来越便利。实践中，我们通常可以用比上一节更简洁的代码来实现同样的模型。在本节中，我们将介绍如何使用tensorflow2.0推荐的keras接口更方便地实现线性回归的训练。

## 3.3.1 生成数据集

我们生成与上一节中相同的数据集。其中`features`是训练数据特征，`labels`是标签。

In [54]:
import tensorflow as tf

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = tf.random.normal(shape=(num_examples, num_inputs),stddev=1)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
# labels 增加随机干扰
labels += tf.random.normal(labels.shape, stddev=0.01)

## 3.3.2 读取数据

虽然tensorflow2.0对于线性回归可以直接拟合，不用再划分数据集，但我们仍学习一下读取数据的方法。

In [55]:
from tensorflow import data as tfdata

batch_size = 10
# 将训练集的特(征和标签组合
dataset = tfdata.Dataset.from_tensor_slices((features, labels))
# 随机读取小批量数据
dataset = dataset.shuffle(buffer_size=num_examples)
dataset = dataset.batch(batch_size)
# iter 方法创建一个迭代器
data_iter = iter(dataset)

`shuffle` 的 `buffer_size` 参数应大于等于样本数，`batch` 可以指定 `batch_size` 的分割大小。

In [25]:
for (batch, (X, y)) in enumerate(dataset):
    print(batch)
    print(X, y)
    break

0
tf.Tensor(
[[ 1.5882721  -0.4413251 ]
 [-0.41925693  0.692152  ]
 [-2.068257   -1.0048193 ]
 [ 0.5290812   0.07397223]
 [ 1.2481225  -0.6947398 ]
 [ 1.4212433   0.4069843 ]
 [ 0.30451813 -0.07301793]
 [-0.09720113  0.6606247 ]
 [ 0.15103829 -0.04567332]
 [ 1.4664663  -0.5699477 ]], shape=(10, 2), dtype=float32) tf.Tensor(
[8.88295   1.0037544 3.4902809 5.0176535 9.049796  5.6652336 5.0645494
 1.7540811 4.6641946 9.056335 ], shape=(10,), dtype=float32)


## 3.3.3 定义模型和初始化参数

`Tensorflow 2.0`推荐使用`Keras`定义网络，故使用`Keras`定义网络
我们先定义一个模型变量`model`，它是一个`Sequential`实例。
在`Keras`中，`Sequential`实例可以看作是一个串联各个层的容器。

在构造模型时，我们在该容器中依次添加层。
当给定输入数据时，容器中的每一层将依次推断下一层的输入尺寸。
重要的一点是，在`Keras`中我们无须指定每一层输入的形状。
线性回归，输入层与输出层等效为一层全连接层`keras.layers.Dense()`。

`Keras` 中初始化参数由 `kernel_initializer` 和 `bias_initializer` 选项分别设置权重和偏置的初始化方式。我们从 `tensorflow` 导入 `initializers` 模块，指定权重参数每个元素将在初始化时随机采样于均值为0、标准差为0.01的正态分布。偏差参数默认会初始化为零。`RandomNormal(stddev=0.01)`指定权重参数每个元素将在初始化时随机采样于均值为0、标准差为0.01的正态分布。偏差参数默认会初始化为零。

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import initializers as init

model = keras.Sequential()
# 偏差初始化为0
model.add(layers.Dense(1, kernel_initializer=init.RandomNormal(stddev=0.01)))

## 3.3.4 定义损失函数

`Tensoflow`在`losses`模块中提供了各种损失函数和自定义损失函数的基类，并直接使用它的均方误差损失作为模型的损失函数。


In [27]:
from tensorflow import losses
loss = losses.MeanSquaredError()

## 3.3.5 定义优化算法

同样，我们也无须自己实现小批量随机梯度下降算法。`tensorflow.keras.optimizers` 模块提供了很多常用的优化算法比如SGD、Adam和RMSProp等。下面我们创建一个用于优化model 所有参数的优化器实例，并指定学习率为0.03的小批量随机梯度下降（SGD）为优化算法。

In [9]:
from tensorflow.keras import optimizers
trainer = optimizers.SGD(learning_rate=0.03)

## 3.3.6 训练模型

在使用`Tensorflow`训练模型时，我们通过调用`tensorflow.GradientTape`记录动态图梯度，执行`tape.gradient`获得动态图中各变量梯度。通过 `model.trainable_variables` 找到需要更新的变量，并用 `trainer.apply_gradients` 更新权重，完成一步训练。

In [52]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for (batch, (X, y)) in enumerate(dataset):
        with tf.GradientTape() as tape:
            l = loss(model(X, training=True), y)
        grads = tape.gradient(l, model.trainable_variables)
        trainer.apply_gradients(zip(grads, model.trainable_variables))
        
    l = loss(model(features), labels)
    print('epoch %d, loss: %f' % (epoch, l))

epoch 1, loss: 0.000247
epoch 2, loss: 0.000103
epoch 3, loss: 0.000102


下面我们分别比较学到的模型参数和真实的模型参数。我们可以通过model的`get_weights()`来获得其权重（`weight`）和偏差（`bias`）。学到的参数和真实的参数很接近。

In [39]:
true_w, model.get_weights()[0]

([2, -3.4], array([[ 1.9994655],
        [-3.40126  ]], dtype=float32))

In [40]:
true_b, model.get_weights()[1]

(4.2, array([4.199513], dtype=float32))

## 小结

- 使用`Tensorflow`可以更简洁地实现模型。
- `tensorflow.data`模块提供了有关数据处理的工具，`tensorflow.keras.layers`模块定义了大量神经网络的层，`tensorflow.initializers`模块定义了各种初始化方法，`tensorflow.optimizers`模块提供了模型的各种优化算法。

In [53]:
help(losses.mean_squared_error)

Help on function mean_squared_error in module tensorflow.python.keras.losses:

mean_squared_error(y_true, y_pred)



In [45]:
help(losses.MeanSquaredError)

Help on class MeanSquaredError in module tensorflow.python.keras.losses:

class MeanSquaredError(LossFunctionWrapper)
 |  MeanSquaredError(reduction='auto', name='mean_squared_error')
 |  
 |  Computes the mean of squares of errors between labels and predictions.
 |  
 |  `loss = square(y_true - y_pred)`
 |  
 |  Usage:
 |  
 |  ```python
 |  mse = tf.keras.losses.MeanSquaredError()
 |  loss = mse([0., 0., 1., 1.], [1., 1., 1., 0.])
 |  print('Loss: ', loss.numpy())  # Loss: 0.75
 |  ```
 |  
 |  Usage with the `compile` API:
 |  
 |  ```python
 |  model = tf.keras.Model(inputs, outputs)
 |  model.compile('sgd', loss=tf.keras.losses.MeanSquaredError())
 |  ```
 |  
 |  Method resolution order:
 |      MeanSquaredError
 |      LossFunctionWrapper
 |      Loss
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, reduction='auto', name='mean_squared_error')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ---------------------------